<a href="https://colab.research.google.com/github/aguspatur22/KNN-Recommendation-Engine/blob/main/KNN_Recommendation_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [112]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-04-04 19:33:43--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M   132MB/s    in 0.2s    

2024-04-04 19:33:44 (132 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [114]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [115]:
# Filtering dataframes
user_counts = df_ratings['user'].value_counts()
book_counts = df_ratings['isbn'].value_counts()

# Remove missing values
df_books.dropna(inplace=True)
df_ratings.dropna(inplace=True)

# Remove from the dataset users with less than 200 ratings and books with less than 100 ratings.
df_ratings = df_ratings[(~df_ratings['user'].isin(user_counts[user_counts < 200].index)) & (~df_ratings['isbn'].isin(book_counts[book_counts < 100].index))]

# Pivot the DataFrame to create a user-item matrix and Fill missing values with 0
user_item_matrix = df_ratings.pivot_table(index=['user'], columns=['isbn'], values='rating').fillna(0)

# Transpose the DataFrame so that books are rows and users are columns
df_ratings = user_item_matrix.T

# Join df_ratings with df_books on 'isbn', select the 'title' column, sort by index, and set this as the new index for df_ratings
df_ratings.index = df_ratings.join(df_books.set_index('isbn'))['title'].sort_index()

In [116]:
KNN_model = NearestNeighbors(metric='cosine')
KNN_model.fit(df_ratings.values)

NearestNeighbors(metric='cosine')

In [123]:
# Function to return recommended books
def get_recommends(book: str = ""):
    title = df_ratings.loc[book]

    distance, indice = KNN_model.kneighbors([title.values], n_neighbors=6)

    recommended_books = [book, pd.DataFrame({'title': df_ratings.iloc[indice[0]].index.values, 'distance': distance[0]}).sort_values(by='distance', ascending=False).head(4).values]

    return recommended_books

In [124]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", array([["I'll Be Seeing You", 0.8016210794448853],
       ['The Weight of Water', 0.7708583474159241],
       ['The Surgeon', 0.7699410915374756],
       ['I Know This Much Is True', 0.7677075266838074]], dtype=object)]
You passed the challenge! 🎉🎉🎉🎉🎉
